## Working with DGGAL in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.vgrid.vn/lab/index.html?path=vgrid/08_dggal.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/main/docs/notebooks/08_dggal.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD?filepath=docs/notebooks/08_dggal.ipynb)
[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeoshub/vgrid/blob/main/docs/notebooks/08_dggal.ipynb)
[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/08_dggal.ipynb)

Full Vgrid DGGS documentation is available at [vgrid document](https://vgrid.gishub.vn).

To work with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/). Full Vgridpandas DGGS documentation is available at [vgridpandas document](https://vgridpandas.gishub.vn).

To work with Vgrid DGGS in QGIS, install the [Vgrid Plugin](https://plugins.qgis.org/plugins/vgridtools/).

To visualize DGGS in Maplibre GL JS, try the [vgrid-maplibre](https://www.npmjs.com/package/vgrid-maplibre) library.

For an interactive demo, visit the [Vgrid Homepage](https://vgrid.vn).

### Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/).

In [ ]:
# %pip install vgrid --upgrade

### latlon2dggal

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2dggal
dggs_type = 'isea9r' # choose one from ['gnosis','isea3h','isea9r','ivea3h',
                    # 'ivea9r','rtea3h','rtea9r','rhealpix']
resolution = 0
dggal_id = latlon2dggal(dggs_type, 10.775276, 106.706797, resolution)
dggal_id

### DGGAL to Shapely Polygon

In [ ]:
from vgrid.conversion.dggs2geo.dggal2geo import dggal2geo
dggal_geo = dggal2geo(dggs_type,dggal_id)
dggal_geo

### DGGAL to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.dggal2geo import dggal2geojson
import json
dggal_geojson = dggal2geojson(dggs_type, dggal_id)
print(json.dumps(dggal_geojson))

### Vector to DGGAL

In [ ]:
from vgrid.conversion.vector2dggs.vector2dggal import vector2dggal

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon2.geojson"
)
dggs_type = "isea9r" # "gnosis", "isea3h", "isea9r", "ivea3h", "ivea9r", "rtea3h", "rtea9r", "rhealpix"
resolution = 9
vector_to_dggal  = vector2dggal(file_path, dggal_type=dggs_type, resolution=resolution, 
                                compact=False, predicate = "intersects",output_format="gpd")
# Visualize the output
vector_to_dggal.plot(edgecolor="white")

### DGGAL Compact

In [ ]:
from vgrid.conversion.dggscompact.dggalcompact import dggalcompact
dggal_compacted = dggalcompact(dggs_type, vector_to_dggal, output_format="gpd")
print(len(dggal_compacted))
dggal_compacted.plot(edgecolor="white")

### DGGAL Expand

In [ ]:
from vgrid.conversion.dggscompact.dggalcompact import dggalexpand

# dggs_type = 'isea3h' # 'gnosis','isea3h','isea9r','ivea3h','ivea9r','rtea3h','rtea9r','rhealpix'
dggal_expanded = dggalexpand(dggs_type,vector_to_dggal, 
                            resolution=10, output_format="gpd")
# dggal_expanded
dggal_expanded.plot(edgecolor="white")

### DGGAL Binning

In [ ]:
from vgrid.binning.dggalbin import dggalbin
dggs_type = 'isea9r' # choose one from ['gnosis','isea3h','isea9r','ivea3h',
                    # 'ivea9r','rtea3h','rtea9r','rhealpix']
file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
stats = "majority"
dggal_bin = dggalbin(file_path,dggs_type=dggs_type, resolution=10, stats=stats, 
                numeric_field="confidence",
                # category="category",
                # method="grid",
                output_format="gpd")
# dggal_bin.head()
dggal_bin.plot(
    column=stats,        # numeric column to base the colors on
    cmap='Spectral_r',        # color scheme (matplotlib colormap)
    legend=True,  
    linewidth=0.2          # boundary width (optional)
)

### Raster to DGGAL

#### Download and open raster

In [ ]:
from vgrid.utils.io import download_file
import rasterio
from rasterio.plot import show

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)
src = rasterio.open(raster_file, 'r')
print(src.meta)
show(src)

#### Convert raster to DGGAL

In [ ]:
from vgrid.conversion.raster2dggs.raster2dggal import raster2dggal
dggs_type = 'isea9r' # 'gnosis','isea3h','isea9r','ivea3h','ivea9r','rtea3h','rtea9r','rhealpix'
raster_to_dggal =  raster2dggal(raster_file, dggs_type=dggs_type, output_format="gpd")

# Visualize the output 
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=28)

dggal_layer = folium.GeoJson(
    raster_to_dggal,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 0.8,
    },
    popup=folium.GeoJsonPopup(
        fields=[f"dggal_{dggs_type}", "band_1", "band_2", "band_3"],
        aliases=["Zone ID", "Band 1", "Band 2", "Band 3"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(dggal_layer.get_bounds())

# Display the map
m

### DGGAL Generator

In [ ]:
from vgrid.generator.dggalgen import dggalgrid   

dggs_type = "isea9r" # "gnosis", "isea3h", "isea9r", "ivea3h", "ivea9r", "rtea3h", "rtea9r", "rhealpix"
resolution = 0
# bbox = (10.762811, 106.699007, 10.778649, 106.717674)  # (min_lat, min_lon, max_lat, max_lon)
dggal_grid = dggalgrid(dggs_type=dggs_type,resolution=resolution, compact=True, output_format="gpd")
# dggal_grid
dggal_grid.plot(edgecolor="white")

### DGGAL Inspect

In [ ]:
from vgrid.stats.dggalstats import dggalinspect
dggs_type = "isea9r" # choose one from ['gnosis','isea3h','isea9r','ivea3h',
                    # 'ivea9r','rtea3h','rtea9r','rhealpix']
resolution = 3
dggal_inspect = dggalinspect(dggs_type, resolution)
dggal_inspect.head()

### Distribution of DGGAL Area Distortions

In [ ]:
from vgrid.stats.dggalstats import dggal_norm_area
dggal_norm_area(dggal_inspect,dggs_type=dggs_type, crs='proj=moll')

### Distribution of DGGAL IPQ Compactness

In [ ]:
from vgrid.stats.dggalstats import dggal_compactness
dggal_compactness(dggal_inspect,dggs_type, crs = "proj=moll")  

### DGGAL Statistics

In [ ]:
from vgrid.stats.dggalstats import dggalstats    
dggal_type = 'rhealpix' # choose one from ['gnosis','isea3h','isea9r','ivea3h',
                    # 'ivea9r','rtea3h','rtea9r','rhealpix']
dggal_stats = dggalstats(dggal_type, unit="km")
dggal_stats